In [8]:
from pde import DiffusionPDE, ScalarField, UnitGrid, CartesianGrid, MemoryStorage, PDE, movie, plot_kymograph
import numpy as np
import gc

In [9]:
def generate_string(params, opp='cos', var='x'):
    string = ""
    for idx, (weight, freq) in enumerate(params):
        if(idx == 0):
            string += f"{weight:.3f}*{opp}({freq:.3f}*2*pi*{var})"
        else:
            string += f"+{weight:.3f}*{opp}({freq:.3f}*2*pi*{var})"
    return string

In [10]:
# generate a set of random periodic initializations
def generate_sinusoidal_weights(cos_x_params, sin_x_params, cos_y_params, sin_y_params):
    cos_x_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(cos_x_params)]
    sin_x_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(sin_x_params)]
    cos_y_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(cos_y_params)]
    sin_y_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(sin_y_params)]
    return cos_x_inital_weights, sin_x_inital_weights, cos_y_inital_weights, sin_y_inital_weights

def generate_sinusoidal_equation(cos_x_inital_weights, sin_x_inital_weights, cos_y_inital_weights, sin_y_inital_weights):
    init_equation = generate_string(cos_x_inital_weights)
    init_equation += '+' + generate_string(sin_x_inital_weights, opp='sin')
    init_equation += '+' + generate_string(cos_y_inital_weights, var='y')
    init_equation += '+' + generate_string(sin_y_inital_weights, opp='sin', var='y')
    return init_equation

cos_x_params = [(0.4, 0.25),(0.3, 0.3)]
sin_x_params = [(0.25, 0.1),(0.0, 0.3),(0.5, 0.1)]
cos_y_params = [(0.1, 0.1),(0.0, 0.1),(0.2, 0.01),(0.3, 0.1)]
sin_y_params = [(0.25, 0.125),]

np.random.seed(1999)
train_eqs = []
for ex in range(100):
    weights = generate_sinusoidal_weights(cos_x_params, sin_x_params, cos_y_params, sin_y_params)
    init_equation = generate_sinusoidal_equation(*weights)
    train_eqs.append(init_equation)

val_eqs = []
for ex in range(15):
    weights = generate_sinusoidal_weights(cos_x_params, sin_x_params, cos_y_params, sin_y_params)
    init_equation = generate_sinusoidal_equation(*weights)
    val_eqs.append(init_equation)

test_eqs = []
for ex in range(15):
    weights = generate_sinusoidal_weights(cos_x_params, sin_x_params, cos_y_params, sin_y_params)
    init_equation = generate_sinusoidal_equation(*weights)
    test_eqs.append(init_equation)

with open('init_periodic_equations.npz', mode='wb') as f:
    np.savez(f, train_eqs=train_eqs, val_eqs=val_eqs, test_eqs=test_eqs)

In [11]:
def generate_random_init():
    init_equation = np.random.uniform(low=0.1, high=0.9, size=(64, 64))
    return init_equation

np.random.seed(1999)
train_eqs = []
for ex in range(100):
    init_equation = generate_random_init()
    train_eqs.append(init_equation)

val_eqs = []
for ex in range(15):
    init_equation = generate_random_init()
    val_eqs.append(init_equation)

test_eqs = []
for ex in range(15):
    init_equation = generate_random_init()
    test_eqs.append(init_equation)

with open('init_uniform_equations.npz', mode='wb') as f:
    np.savez(f, train_eqs=train_eqs, val_eqs=val_eqs, test_eqs=test_eqs)

In [ ]:
np.random.seed(1999)
train_eqs = []
for ex in range(100):
    init_equation = np.random.uniform(low=0.1, high=0.4)
    train_eqs.append(init_equation)

val_eqs = []
for ex in range(15):
    init_equation = np.random.uniform(low=0.1, high=0.4)
    val_eqs.append(init_equation)

test_eqs = []
for ex in range(15):
    init_equation = np.random.uniform(low=0.1, high=0.4)
    test_eqs.append(init_equation)

with open('init_nu_equations.npz', mode='wb') as f:
    np.savez(f, train_eqs=train_eqs, val_eqs=val_eqs, test_eqs=test_eqs)

In [ ]:
def generate_grid(N:int, x_periodic:bool, y_periodic:bool):
    grid = UnitGrid([N, N], periodic=[x_periodic, y_periodic])
    return grid

def generate_equation_state(grid, init_equation):
    state = ScalarField.from_expression(grid, init_equation)
    state.data = np.maximum(np.minimum(state.data, 0.9), 0.1)
    return state

def generate_data_state(grid, data):
    state = ScalarField(grid, data=data)
    return state

def get_nu_sample():
    nu = np.random.uniform(low=0.1, high=0.4)
    nu = round(nu, 4)
    print(nu)
    return nu

def get_fixed_nu():
    return 0.2

def generate_boundary_conditions(x_periodic:bool, y_periodic:bool):
    if(x_periodic):
        bc_x = 'periodic'
    else:
        bc_x = {"derivative": 0.0}
    if(y_periodic):
        bc_y = 'periodic'
    else:
        bc_y = {"derivative": 0.0}
    return [bc_x, bc_y]

def get_burgers_pde(nu, bcs):
    eq = PDE(
        {'u':f'{nu} * (laplace(u) + laplace(u)) - u * (getx(gradient(u)) + gety(gradient(u)))'},
        user_funcs={'getx':lambda arr: arr[0], 'gety':lambda arr: arr[1]},
        bc=bcs
    )
    return eq

def solve_pde(eq, state):
    example = []
    storage = MemoryStorage()
    results = eq.solve(state, t_range=10, dt=0.25, tracker=["progress", storage.tracker(0.25)])
    for time_step, field in storage.items():
        example.append(np.expand_dims(field.data, axis=0))
    example = np.expand_dims(np.concatenate(example.copy()), axis=0)
    # movie(storage, filename="example_burgers.mov")
    print(example.min(), example.max(), example.mean(), example.std())
    if(np.any(np.isnan(example))):
        raise Exception('')
    return example

def generate_example(N, x_periodic, y_periodic, init_equation:str=None, random_data:np.array=None, random_nu_sample:bool=True):
    # get the grid
    grid = generate_grid(N, x_periodic, y_periodic)
    # get the initial state
    if(init_equation is not None):
        state = generate_equation_state(grid, init_equation)
    elif(random_data is not None):
        init_equation = 'uniform 0.1 to 0.9'
        state = generate_data_state(grid, random_data)
    else:
        raise Exception('we need an equation or data')
    # get the hyperparams
    if(random_nu_sample):
        nu = get_nu_sample()
    else:
        nu = get_fixed_nu()
    # get boundary conditions
    boundary_conditions = generate_boundary_conditions(x_periodic, y_periodic)
    # get the pde
    eq = get_burgers_pde(nu, boundary_conditions)
    # solve the pde
    solution = solve_pde(eq, state)
    # delete the things we don't need
    del grid
    del state
    del eq
    gc.collect()
    return solution, nu, init_equation



In [6]:
def generate_grid(N:int, x_periodic:bool, y_periodic:bool):
    grid = UnitGrid([N, N], periodic=[x_periodic, y_periodic])
    return grid

def generate_sinusoidal_weights(cos_x_params, sin_x_params, cos_y_params, sin_y_params):
    cos_x_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(cos_x_params)]
    sin_x_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(sin_x_params)]
    cos_y_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(cos_y_params)]
    sin_y_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(sin_y_params)]
    return cos_x_inital_weights, sin_x_inital_weights, cos_y_inital_weights, sin_y_inital_weights

def generate_sinusoidal_equation(cos_x_inital_weights, sin_x_inital_weights, cos_y_inital_weights, sin_y_inital_weights):
    init_equation = generate_string(cos_x_inital_weights)
    init_equation += '+' + generate_string(sin_x_inital_weights, opp='sin')
    init_equation += '+' + generate_string(cos_y_inital_weights, var='y')
    init_equation += '+' + generate_string(sin_y_inital_weights, opp='sin', var='y')
    return init_equation

def generate_periodic_init_state(grid, init_equation):
    state = ScalarField.from_expression(grid, init_equation)
    state.data = np.maximum(np.minimum(state.data, 0.9), 0.1)
    return state

def generate_random_init_state(grid):
    state = ScalarField.random_uniform(grid, 0.1, 0.9)
    return state

def get_nu_sample():
    nu = np.random.uniform(low=0.1, high=0.4)
    print(nu)
    return nu

def get_fixed_nu():
    return 0.2

def generate_boundary_conditions(x_periodic:bool, y_periodic:bool):
    if(x_periodic):
        bc_x = 'periodic'
    else:
        bc_x = {"derivative": 0.0}
    if(y_periodic):
        bc_y = 'periodic'
    else:
        bc_y = {"derivative": 0.0}
    return [bc_x, bc_y]

def get_burgers_pde(nu, bcs):
    eq = PDE(
        {'u':f'{nu} * (laplace(u) + laplace(u)) - u * (getx(gradient(u)) + gety(gradient(u)))'},
        user_funcs={'getx':lambda arr: arr[0], 'gety':lambda arr: arr[1]},
        bc=bcs
    )
    return eq

def solve_pde(eq, state):
    example = []
    storage = MemoryStorage()
    results = eq.solve(state, t_range=10, dt=0.25, tracker=["progress", storage.tracker(0.25)])
    for time_step, field in storage.items():
        example.append(np.expand_dims(field.data, axis=0))
    example = np.expand_dims(np.concatenate(example.copy()), axis=0)
    # movie(storage, filename="example_burgers.mov")
    print(example.min(), example.max(), example.mean(), example.std())
    if(np.any(np.isnan(example))):
        raise Exception('')
    return example


cos_x_params = [(0.4, 0.25),(0.3, 0.3)]
sin_x_params = [(0.25, 0.1),(0.0, 0.3),(0.5, 0.1)]
cos_y_params = [(0.1, 0.1),(0.0, 0.1),(0.2, 0.01),(0.3, 0.1)]
sin_y_params = [(0.25, 0.125),]

def generate_example(N, x_periodic, y_periodic, random_nu_sample:bool=True, periodic_init:bool=True):
    # get the grid
    grid = generate_grid(N, x_periodic, y_periodic)
    # get the initial state
    if(periodic_init):
        weights = generate_sinusoidal_weights(cos_x_params, sin_x_params, cos_y_params, sin_y_params)
        init_equation = generate_sinusoidal_equation(*weights)
        state = generate_periodic_init_state(grid, init_equation)
    else:
        init_equation = 'uniform 0.1 to 0.4'
        state = generate_random_init_state(grid)
    # get the hyperparams
    if(random_nu_sample):
        nu = get_nu_sample()
    else:
        nu = get_fixed_nu()
    # get boundary conditions
    boundary_conditions = generate_boundary_conditions(x_periodic, y_periodic)
    # get the pde
    eq = get_burgers_pde(nu, boundary_conditions)
    # solve the pde
    solution = solve_pde(eq, state)
    # delete the things we don't need
    del grid
    del state
    del eq
    gc.collect()
    return solution, [nu, init_equation]

N = 64
X_PERIOD = False
Y_PERIOD = False
RANDOM_NU = False 
PERIODIC_INIT = True

def generate_all_examples(n=50):
    data, meta = [], []
    for idx in range(n):
        solution, info = generate_example(N, X_PERIOD, Y_PERIOD, random_nu_sample=RANDOM_NU, periodic_init=PERIODIC_INIT)
        data.append(solution)
        meta.append(info)
    return np.concatenate(data), np.concatenate(meta)

np.random.seed(1999)
train_data, train_meta = generate_all_examples(n=50)
val_data, val_meta = generate_all_examples(n=8)
test_data, test_meta = generate_all_examples(n=8)

name = f'burgers_{N}'
if(PERIODIC_INIT):
    name += '_period_init'
else:
    name += '_non_period_init'

if(X_PERIOD):
    name += '_x_period'
else:
    name += '_x_non_period'

if(Y_PERIOD):
    name += '_y_period'
else:
    name += '_y_non_period'

if(RANDOM_NU):
    name += '_varying_nu'
else:
    name += "_fixed_nu"

name += '.npy'
print(name)

with open(name, mode='wb') as f:
    np.save(f, train_data, allow_pickle=True)
    np.save(f, val_data, allow_pickle=True)
    np.save(f, test_data, allow_pickle=True)

    np.save(f, train_meta, allow_pickle=True)
    np.save(f, val_meta, allow_pickle=True)
    np.save(f, test_meta, allow_pickle=True)
    
train_data.shape, val_data.shape, test_data.shape

  0%|          | 0/10.0 [00:00<?, ?it/s]

0.0 1.032492993248362 0.2543676180757367 0.3617299320992598


  0%|          | 0/10.0 [00:00<?, ?it/s]

0.0 1.0275079415147639 0.2885463827766136 0.3136936394344018


KeyboardInterrupt: 

In [ ]:
def generate_example(cos_x_params, sin_x_params, cos_y_params, sin_y_params, diffusivity=0.4):
    N = 64
    grid = UnitGrid([N, N], periodic=[False, False])
    # get random init weights
    cos_x_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(cos_x_params)]
    sin_x_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(sin_x_params)]
    cos_y_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(cos_y_params)]
    sin_y_inital_weights = [(np.random.normal(mu, sigma), (idx + 1) / N) for idx, (mu, sigma) in enumerate(sin_y_params)]
    # get the string equation
    init_equation = generate_string(cos_x_inital_weights)
    init_equation += '+' + generate_string(sin_x_inital_weights, opp='sin')
    init_equation += '+' + generate_string(cos_y_inital_weights, var='y')
    init_equation += '+' + generate_string(sin_y_inital_weights, opp='sin', var='y')
    # generate the inital state
    state = ScalarField.from_expression(grid, init_equation)
    # bc_x = 'periodic'
    # bc_y = 'periodic' 
    bc_x = {"derivative": 0.0}
    bc_y = {"derivative": 0.0}
    eq = DiffusionPDE(diffusivity=diffusivity, bc=[bc_x, bc_y])
    # get examples
    example = []
    storage = MemoryStorage()
    results = eq.solve(state, t_range=4, dt=0.1, tracker=["progress", storage.tracker(0.1)])
    del grid
    del state
    del eq
    gc.collect()

    for time_step, field in storage.items():
        example.append(np.expand_dims(field.data, axis=0))
    example = np.expand_dims(np.concatenate(example.copy()), axis=0)

    del storage
    del results
    gc.collect()
    return example, init_equation

cos_x_params = [(1.0, 1.0),(-0.75, 1.0)]
sin_x_params = [(-0.25, 1.0),(0.0, 0.1),(2.0, 1.0)]
cos_y_params = [(0.0, 0.1),(0.0, 0.1),(0.0, 0.01),(3.0, 1.0)]
sin_y_params = [(2.0, 1.25),]

def get_diffusivity_sample():
    diffusivity = np.random.normal(loc=0.4, scale=0.1)
    diffusivity = max(diffusivity, 0.2)
    diffusivity = min(diffusivity, 0.6)
    return diffusivity

def generate_example_solution(data, meta_info):
    diffusivity = get_diffusivity_sample()
    example, equation = generate_example(cos_x_params, sin_x_params, cos_y_params, sin_y_params, diffusivity=diffusivity)
    data.append(example)
    meta_info.append((diffusivity, equation))

train_data, train_meta = [], []
for train_idx in range(50):
    generate_example_solution(train_data, train_meta)
train_data = np.concatenate(train_data)
train_meta = np.concatenate(train_meta)

val_data, val_meta = [], []
for val_idx in range(8):
    generate_example_solution(val_data, val_meta)
val_data = np.concatenate(val_data)
val_meta = np.concatenate(val_meta)

test_data, test_meta = [], []
for test_idx in range(8):
    generate_example_solution(test_data, test_meta)
test_data = np.concatenate(test_data)
test_meta = np.concatenate(test_meta)

with open('diffusion_varying_sinusoidal_init_varying_diffusivity_non_periodic_boundaries.npy', mode='wb') as f:
    np.save(f, train_data, allow_pickle=True)
    np.save(f, val_data, allow_pickle=True)
    np.save(f, test_data, allow_pickle=True)

    np.save(f, train_meta, allow_pickle=True)
    np.save(f, val_meta, allow_pickle=True)
    np.save(f, test_meta, allow_pickle=True)
    

train_data.shape, val_data.shape, test_data.shape

In [ ]:
grid = UnitGrid([64, 64], periodic=[True, True])
# state = ScalarField.random_uniform(grid, 0.0, 1.0)
state = ScalarField.from_expression(grid, '0.197*cos(0.016*2*pi*x)+-0.917*cos(0.031*2*pi*x)+1.296*sin(0.016*2*pi*x)+-0.107*sin(0.031*2*pi*x)+1.053*sin(0.047*2*pi*x)+-0.012*cos(0.016*2*pi*y)+-0.016*cos(0.031*2*pi*y)+-0.004*cos(0.047*2*pi*y)+3.506*cos(0.062*2*pi*y)+4.199*sin(0.016*2*pi*y)')
state.data = (state.data - state.data.min()) / (state.data.max() - state.data.min())
nu = 0.4
eq = PDE(
    {'u':f'{nu} * (laplace(u) + laplace(u)) - u * (getx(gradient(u)) + gety(gradient(u)))'},
    user_funcs={'getx':lambda arr: arr[0], 'gety':lambda arr: arr[1]}
)

example = []
storage = MemoryStorage()
results = eq.solve(state, t_range=5, dt=0.1, tracker=["progress", storage.tracker(0.1)])

movie(storage, filename="example_burgers.mov")
plot_kymograph(storage)


In [ ]:

grid = UnitGrid([64, 64], periodic=[False, True])  # generate grid
state = ScalarField.random_uniform(grid, 0.0, 1.0)  # generate initial condition

bc_x_left = {"derivative": 0.0}
bc_x_right = {"value": 0.0}
bc_x = [bc_x_left, bc_x_right]
bc_y = "periodic"

eq = DiffusionPDE(diffusivity=0.4, bc=[bc_x, bc_y])

storage = MemoryStorage()
results = eq.solve(state, t_range=10, dt=0.1, tracker=["progress", storage.tracker(0.1)])

In [ ]:
import numpy as np
np.pi * 2

In [ ]:
def generate_string(params, opp='cos', var='x'):
    string = ""
    for idx, param in enumerate(params):
        if(idx == 0):
            string += f"{param}*{opp}({var})"
        else:
            string += f"+{param}*{opp}({var})"
    return string

In [ ]:
def generate_unit_grid(x_dims, y_dims, x_periodic=False, y_periodic=False):
    grid = UnitGrid([x_dims, y_dims], periodic=[x_periodic, y_periodic])
    return grid

def generate_init_field_from_eq(grid, equation):
    state = ScalarField.from_expression(grid, equation)
    return state

def generate_init_field_from_random_uniform(grid, lower, upper):
    state = ScalarField.random_uniform(grid, 0.0, 1.0)
    return state

def generate_init_field_from_random_normal(grid, mu, sigma):
    state = ScalarField.random_normal(grid, mean=mu, scaling=sigma)
    return state

In [ ]:
def generate_boundary_conditions()

In [ ]:
from pde import DiffusionPDE, ScalarField, UnitGrid, CartesianGrid, MemoryStorage

grid = UnitGrid([64, 64], periodic=[False, True])  # generate grid
state = ScalarField.from_expression(grid, "0.5*cos(x)-0.25*cos(y)")  # generate initial condition

bc_x_left = {"derivative": 0.0}
bc_x_right = {"value": 0.0}
bc_x = [bc_x_left, bc_x_right]
bc_y = "periodic"

eq = DiffusionPDE(diffusivity=0.4, bc=[bc_x, bc_y])

storage = MemoryStorage()
results = eq.solve(state, t_range=10, dt=0.1, tracker=["progress", storage.tracker(0.1)])

In [ ]:
for time, field in storage.items():
    print(time, field)

In [ ]:
MemoryStorage?

In [ ]:
results = eq.solve(state, t_range=2, dt=0.1)
results

In [ ]:
results1

In [ ]:
results2

In [ ]:

# set boundary conditions `bc` for all axes
bc_x_left = {"derivative": 0.1}
bc_x_right = {"value": "sin(y / 2)"}
bc_x = [bc_x_left, bc_x_right]
bc_y = "periodic"
eq = DiffusionPDE(bc=[bc_x, bc_y])

result = eq.solve(state, t_range=10, dt=0.005)
# results
result

In [ ]:
import autograd.numpy as np
from autograd import grad, hessian

In [ ]:
def init_conditions(x, y):
    t = np.cos(x) + np.cos(y)
    return t

In [ ]:
grad_init = hessian(init_conditions)
grad_init(0.0, 0.0)

In [ ]:
import numpy as np

In [ ]:
x = np.linspace(start=0, stop=1, num=128)
y = np.linspace(start=0, stop=1, num=128)
x, y = np.meshgrid(x, y)

In [ ]:
x, y = np.meshgrid()

In [ ]:
x = np.random.uniform(low=0, high=1, size=100)
x

In [ ]:
def step_in_time(x):
    

In [ ]:
def next_(u):
    0.4 * (
        np.roll(u, 1, axis=0) + 
        np.roll(u, -1, axis=0) + 
        np.roll(u, 1, axis=1) + 
        np.roll(u, -1, axis=1) -
        4 * u
    ) + u
    

In [ ]:
x = torch.tensor(np.pi / 4.1, requires_grad=True) #np.linspace(start=0, stop=2*np.pi, num=100), requires_grad=True)
y = torch.cos(x)
y.backward()
x.grad

In [ ]:
import deepxde as dde

In [ ]:
ALPHA = 0.01

def pde(X, T):
    dT_xx = dde.grad.hessian(T, X, j=0)
    dT_yy = dde.grad.hessian(T, X, j=1)
    dT_t = dde.grad.jacobian(T, X, j=2)
    return dT_t - (ALPHA * (dT_xx + dT_yy))

In [ ]:
# Define the initial condition
def initial_condition(x, y):
    return np.exp(-(x**2 + y**2))

In [ ]:
# Define the geometry
geom = dde.geometry.Rectangle([0, 0], [1, 1])
geom

In [ ]:
timedomain = dde.geometry.TimeDomain(0, 50)
timedomain

In [ ]:
geomtime = dde.geometry.GeometryXTime(geom, timedomain)
geomtime

In [ ]:
T0 = 300
Tinit = 1000

def func(x):
    return np.ones((len(x),1), dtype=np.float32)*T0

def func_n(x):
    return np.zeros((len(x),1), dtype=np.float32)

def func_init(x):
    return np.ones((len(x),1), dtype=np.float32)*Tinit

In [ ]:
def boundary_x_l(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0)

def boundary_x_r(x, on_boundary):
    return on_boundary and np.isclose(x[0], 1)

def boundary_y_b(x, on_boundary):
    return on_boundary and np.isclose(x[1], 0)

def boundary_y_u(x, on_boundary):
    return on_boundary and np.isclose(x[1], 1)

In [ ]:
bc_x_l = dde.DirichletBC(geomtime, func, boundary_x_l)
bc_x_r = dde.DirichletBC(geomtime, func, boundary_x_r)
bc_y_b = dde.DirichletBC(geomtime, func, boundary_y_b)
bc_y_u = dde.NeumannBC(geomtime, func_n, boundary_y_u)
ic = dde.IC(geomtime, func_init, lambda _, on_initial: on_initial)

In [ ]:
data = dde.data.TimePDE(
    geomtime,
    pde, 
    [bc_x_l, bc_x_r, bc_y_b, bc_y_u, ic],
    num_domain=1000,
    num_boundary=1000,
    num_initial=1000,
)

In [ ]:
data.train_next_batch()
data.


In [ ]:
data.test()

In [ ]:
net = dde.nn.FNN([1000], "tanh", "Glorot normal")

In [ ]:
model = dde.Model(data, net)
model.compile("adam", lr=1e-3)

In [ ]:
losshistory, train_state = model.train(iterations=20000)

In [ ]:
data.update()

In [ ]:
data.get_x()

In [ ]:

initial_condition_data = dde.data.PDE(geom, heat_equation, lambda _, on_initial: initial_condition(*on_initial), num_domain=1000, num_boundary=100)

In [ ]:
x = np.linspace(start=0, stop=2*np.pi, num=100)
y = np.cos(x)

dde.grad.jacobian(x, y)